# CHEAQI Spatial Interpolation Workbench
This notebook provides an interactive workflow for transforming CSV environmental observations into gridded rasters.

In [1]:
import json
import math
import os
import shutil
import subprocess
import tempfile
import traceback
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

import ipywidgets as widgets
import numpy as np
import pandas as pd
from IPython.display import clear_output, display
from pyproj import CRS, Transformer
from tqdm.auto import tqdm

try:
    from osgeo import gdal, osr
    HAS_GDAL_PY = True
except Exception:
    gdal = None
    osr = None
    HAS_GDAL_PY = False

try:
    from scipy.spatial import cKDTree
    HAS_SCIPY = True
except Exception:
    cKDTree = None
    HAS_SCIPY = False

try:
    from pykrige.ok import OrdinaryKriging
    HAS_PYKRIGE = True
except Exception:
    OrdinaryKriging = None
    HAS_PYKRIGE = False

try:
    import fiona
    HAS_FIONA = True
except Exception:
    fiona = None
    HAS_FIONA = False

try:
    import rasterio
    from rasterio.transform import Affine
    HAS_RASTERIO = True
except Exception:
    rasterio = None
    Affine = None
    HAS_RASTERIO = False

try:
    import xarray as xr
    HAS_XARRAY = True
except Exception:
    xr = None
    HAS_XARRAY = False

GDAL_CMDS: Dict[str, Optional[str]] = {
    "gdal_grid": shutil.which("gdal_grid"),
    "gdalbuildvrt": shutil.which("gdalbuildvrt"),
    "gdal_translate": shutil.which("gdal_translate"),
}
GDAL_AVAILABLE = all(GDAL_CMDS.values())

NODATA_VALUE = -9999.0
MASTER_STACK_NAME = "CHEAQI_master_stack.tif"
NETCDF_TIMESERIES_NAME = "CHEAQI_timeseries.nc"

pd.options.mode.chained_assignment = None


In [2]:
def determine_utm_epsg(lon: float, lat: float) -> int:
    zone = int((lon + 180.0) / 6.0) + 1
    return 32600 + zone if lat >= 0 else 32700 + zone


def prepare_dataframe(
    df: pd.DataFrame,
    lon_col: str,
    lat_col: str,
    date_col: str,
    variable_cols: Sequence[str],
) -> pd.DataFrame:
    working = df.copy()
    working[lon_col] = pd.to_numeric(working[lon_col], errors="coerce")
    working[lat_col] = pd.to_numeric(working[lat_col], errors="coerce")
    working["__cheaqi_date"] = pd.to_datetime(working[date_col], errors="coerce", utc=False)
    working["__cheaqi_date"] = working["__cheaqi_date"].dt.tz_localize(None)
    working = working.dropna(subset=[lon_col, lat_col, "__cheaqi_date"])
    if working.empty:
        raise ValueError("No valid rows remain after cleaning coordinates and dates.")
    working = working.copy()
    for column in variable_cols:
        working[column] = pd.to_numeric(working[column], errors="coerce")
    value_mask = working[variable_cols].notna().any(axis=1)
    working = working.loc[value_mask]
    if working.empty:
        raise ValueError("Selected variables contain no numeric samples.")
    return working


def reproject_dataframe(df: pd.DataFrame, lon_col: str, lat_col: str, target_epsg: int) -> pd.DataFrame:
    transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{target_epsg}", always_xy=True)
    xs, ys = transformer.transform(df[lon_col].to_numpy(), df[lat_col].to_numpy())
    result = df.copy()
    result["x"] = xs
    result["y"] = ys
    return result


def read_aoi_bounds(aoi_path: Optional[str], target_epsg: int) -> Optional[Tuple[float, float, float, float]]:
    if not aoi_path:
        return None
    path = Path(aoi_path).expanduser()
    if not path.exists():
        raise FileNotFoundError(f"AOI file not found: {path}")
    target = CRS.from_epsg(int(target_epsg))
    if HAS_FIONA:
        with fiona.open(path) as src:
            bounds = src.bounds  # minx, miny, maxx, maxy
            if not src.crs:
                raise ValueError("AOI CRS is undefined.")
            source = CRS(src.crs)
            if source == target:
                return bounds
            transformer = Transformer.from_crs(source, target, always_xy=True)
            minx, miny = transformer.transform(bounds[0], bounds[1])
            maxx, maxy = transformer.transform(bounds[2], bounds[3])
            return (min(minx, maxx), min(miny, maxy), max(minx, maxx), max(miny, maxy))
    if HAS_GDAL_PY:
        datasource = gdal.OpenEx(str(path), gdal.OF_VECTOR)
        if datasource is None:
            raise RuntimeError("Unable to read AOI with GDAL.")
        layer = datasource.GetLayer(0)
        extent = layer.GetExtent()  # minx, maxx, miny, maxy
        spatial_ref = layer.GetSpatialRef()
        datasource = None
        if spatial_ref is None:
            raise ValueError("AOI CRS is undefined.")
        source = CRS.from_wkt(spatial_ref.ExportToWkt())
        if source == target:
            return (extent[0], extent[2], extent[1], extent[3])
        transformer = Transformer.from_crs(source, target, always_xy=True)
        minx, miny = transformer.transform(extent[0], extent[2])
        maxx, maxy = transformer.transform(extent[1], extent[3])
        return (min(minx, maxx), min(miny, maxy), max(minx, maxx), max(miny, maxy))
    raise RuntimeError("Reading an AOI requires either Fiona or GDAL Python bindings.")


def derive_extent(
    df: pd.DataFrame,
    cell_size: float,
    aoi_path: Optional[str],
    target_epsg: int,
) -> Tuple[float, float, float, float]:
    extent = None
    if aoi_path:
        try:
            extent = read_aoi_bounds(aoi_path, target_epsg)
            if extent:
                rounded = [round(val, 2) for val in extent]
                print(f"Using AOI-derived extent: {rounded}")
        except Exception as exc:
            print(f"AOI extent unavailable ({exc}); falling back to point coverage.")
    if extent is None:
        xmin = float(df["x"].min())
        xmax = float(df["x"].max())
        ymin = float(df["y"].min())
        ymax = float(df["y"].max())
        buffer_size = max(cell_size, cell_size * 0.5)
        extent = (xmin - buffer_size, ymin - buffer_size, xmax + buffer_size, ymax + buffer_size)
        print("Point-derived extent (with buffer) will be used.")
    if extent[0] >= extent[2] or extent[1] >= extent[3]:
        raise ValueError("Extent is invalid; please verify inputs.")
    return extent


def build_grid(extent: Tuple[float, float, float, float], cell_size: float) -> Dict[str, np.ndarray]:
    xmin, ymin, xmax, ymax = extent
    width = max(1, int(math.ceil((xmax - xmin) / cell_size)))
    height = max(1, int(math.ceil((ymax - ymin) / cell_size)))
    xmax_adjusted = xmin + width * cell_size
    ymax_adjusted = ymin + height * cell_size
    transform = (xmin, cell_size, 0.0, ymax_adjusted, 0.0, -cell_size)
    x_centers = xmin + cell_size * (np.arange(width) + 0.5)
    y_centers = ymax_adjusted - cell_size * (np.arange(height) + 0.5)
    grid_x, grid_y = np.meshgrid(x_centers, y_centers)
    return {
        "width": width,
        "height": height,
        "transform": transform,
        "grid_x": grid_x,
        "grid_y": grid_y,
        "x_coords": x_centers,
        "y_coords": y_centers,
    }


def create_vrt(csv_path: Path, layer_name: str, epsg: int, x_field: str = "x", y_field: str = "y") -> str:
    sanitized = Path(csv_path).as_posix()
    return f'''<OGRVRTDataSource>
  <OGRVRTLayer name="{layer_name}">
    <SrcDataSource>{sanitized}</SrcDataSource>
    <GeometryType>wkbPoint</GeometryType>
    <LayerSRS>EPSG:{epsg}</LayerSRS>
    <GeometryField encoding="PointFromColumns" x="{x_field}" y="{y_field}"/>
  </OGRVRTLayer>
</OGRVRTDataSource>
'''


def run_command(command: Sequence[str], env: Optional[Dict[str, str]] = None) -> None:
    process = subprocess.run(command, capture_output=True, text=True, env=env)
    if process.stdout:
        print(process.stdout.strip())
    if process.returncode != 0:
        stderr = process.stderr.strip()
        raise RuntimeError(f"Command failed ({command[0]}): {stderr or process.stdout}")


In [3]:
def set_band_descriptions(raster_path: Path, band_names: Sequence[str]) -> None:
    if not band_names:
        return
    raster_path = Path(raster_path)
    applied = False
    if HAS_GDAL_PY:
        dataset = gdal.Open(str(raster_path), gdal.GA_Update)
        if dataset is not None:
            for index, name in enumerate(band_names, start=1):
                band = dataset.GetRasterBand(index)
                if band is not None:
                    band.SetDescription(str(name))
            dataset.FlushCache()
            dataset = None
            applied = True
    if not applied and HAS_RASTERIO:
        try:
            with rasterio.open(raster_path, "r+") as dst:
                for index, name in enumerate(band_names, start=1):
                    dst.set_band_description(index, str(name))
            applied = True
        except Exception:
            pass
    if not applied:
        print(f"Warning: unable to set band names for {raster_path}")


def write_geotiff_stack(
    stack: np.ndarray,
    transform: Tuple[float, float, float, float, float, float],
    epsg: int,
    output_path: Path,
    band_names: Optional[Sequence[str]] = None,
    nodata: float = NODATA_VALUE,
) -> None:
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    bands, height, width = stack.shape
    band_names = list(band_names or [])
    if HAS_GDAL_PY:
        driver = gdal.GetDriverByName("GTiff")
        dataset = driver.Create(
            str(output_path),
            width,
            height,
            bands,
            gdal.GDT_Float32,
            options=["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=IF_SAFER"],
        )
        dataset.SetGeoTransform(transform)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(int(epsg))
        dataset.SetProjection(srs.ExportToWkt())
        for index in range(bands):
            band = dataset.GetRasterBand(index + 1)
            data = stack[index].astype(np.float32)
            mask = ~np.isfinite(data)
            if mask.any():
                data = data.copy()
                data[mask] = nodata
            band.WriteArray(data)
            band.SetNoDataValue(nodata)
            if index < len(band_names):
                band.SetDescription(str(band_names[index]))
        dataset.FlushCache()
        dataset = None
        return
    if HAS_RASTERIO and Affine is not None:
        affine = Affine(transform[1], transform[2], transform[0], transform[4], transform[5], transform[3])
        with rasterio.open(
            output_path,
            "w",
            driver="GTiff",
            height=height,
            width=width,
            count=bands,
            dtype="float32",
            crs=f"EPSG:{epsg}",
            transform=affine,
            nodata=nodata,
            compress="DEFLATE",
            tiled=True,
            BIGTIFF="IF_SAFER",
        ) as dst:
            for index in range(bands):
                data = stack[index].astype(np.float32)
                mask = ~np.isfinite(data)
                if mask.any():
                    data = data.copy()
                    data[mask] = nodata
                dst.write(data, index + 1)
                if index < len(band_names):
                    dst.set_band_description(index + 1, str(band_names[index]))
        return
    raise RuntimeError("Writing GeoTIFF rasters requires GDAL or rasterio.")


def idw_interpolate(
    coords: np.ndarray,
    values: np.ndarray,
    grid_x: np.ndarray,
    grid_y: np.ndarray,
    power: float,
    workers: int,
) -> np.ndarray:
    if coords.size == 0:
        return np.full_like(grid_x, np.nan, dtype=np.float32)
    tree = cKDTree(coords)
    samples = np.column_stack([grid_x.ravel(), grid_y.ravel()])
    neighbors = min(coords.shape[0], 12)
    distances, indices = tree.query(samples, k=neighbors, workers=max(1, workers))
    if neighbors == 1:
        distances = distances[:, np.newaxis]
        indices = indices[:, np.newaxis]
    zero_mask = distances == 0
    results = np.empty(samples.shape[0], dtype=np.float32)
    if zero_mask.any():
        zero_rows = zero_mask.any(axis=1)
        zero_indices = zero_mask[zero_rows].argmax(axis=1)
        results[zero_rows] = values[indices[zero_rows, zero_indices]]
    else:
        zero_rows = np.zeros(samples.shape[0], dtype=bool)
    remaining = ~zero_rows
    if remaining.any():
        weights = np.where(distances[remaining] == 0, 0.0, 1.0 / np.power(distances[remaining], power))
        weight_sum = weights.sum(axis=1)
        estimates = np.full(weight_sum.shape, np.nan, dtype=np.float32)
        valid = weight_sum > 0
        if valid.any():
            weighted = weights[valid] * values[indices[remaining][valid]]
            estimates[valid] = weighted.sum(axis=1) / weight_sum[valid]
        results[remaining] = estimates
    return results.reshape(grid_x.shape)


def create_master_stack(band_plan: Sequence[Dict[str, str]], output_path: Path) -> Optional[Path]:
    band_plan = list(band_plan)
    if not band_plan:
        return None
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    template_path = Path(band_plan[0]["path"])
    if HAS_GDAL_PY:
        template = gdal.Open(str(template_path))
        if template is None:
            raise RuntimeError(f"Unable to open template raster: {template_path}")
        width = template.RasterXSize
        height = template.RasterYSize
        transform = template.GetGeoTransform()
        projection = template.GetProjection()
        driver = gdal.GetDriverByName("GTiff")
        dataset = driver.Create(
            str(output_path),
            width,
            height,
            len(band_plan),
            gdal.GDT_Float32,
            options=["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=IF_SAFER"],
        )
        dataset.SetGeoTransform(transform)
        if projection:
            dataset.SetProjection(projection)
        for index, plan in enumerate(band_plan, start=1):
            source = gdal.Open(str(plan["path"]))
            if source is None:
                raise RuntimeError(f"Unable to open {plan['path']} for master stack.")
            src_band = source.GetRasterBand(int(plan["band"]))
            data = src_band.ReadAsArray().astype(np.float32)
            mask = ~np.isfinite(data)
            if mask.any():
                data = data.copy()
                data[mask] = NODATA_VALUE
            dst_band = dataset.GetRasterBand(index)
            dst_band.WriteArray(data)
            dst_band.SetNoDataValue(NODATA_VALUE)
            dst_band.SetDescription(plan.get("label", ""))
            source = None
        dataset.FlushCache()
        dataset = None
        template = None
        return output_path
    if HAS_RASTERIO:
        with rasterio.open(template_path) as template:
            profile = template.profile
        profile.update(
            count=len(band_plan),
            dtype="float32",
            compress="DEFLATE",
            tiled=True,
            BIGTIFF="IF_SAFER",
            nodata=NODATA_VALUE,
        )
        with rasterio.open(output_path, "w", **profile) as dst:
            for index, plan in enumerate(band_plan, start=1):
                with rasterio.open(plan["path"]) as src:
                    data = src.read(int(plan["band"])).astype(np.float32)
                mask = ~np.isfinite(data)
                if mask.any():
                    data = data.copy()
                    data[mask] = NODATA_VALUE
                dst.write(data, index)
                dst.set_band_description(index, plan.get("label", ""))
        return output_path
    if GDAL_AVAILABLE:
        with tempfile.TemporaryDirectory() as tmpdir:
            tmpdir = Path(tmpdir)
            vrt_path = tmpdir / "master_stack.vrt"
            ordered_files: List[str] = []
            for entry in band_plan:
                path = entry["path"]
                if not ordered_files or ordered_files[-1] != path:
                    ordered_files.append(path)
            command = [GDAL_CMDS["gdalbuildvrt"], "-separate", str(vrt_path), *ordered_files]
            run_command(command)
            translate_command = [
                GDAL_CMDS["gdal_translate"],
                str(vrt_path),
                str(output_path),
                "-co",
                "COMPRESS=DEFLATE",
                "-co",
                "TILED=YES",
                "-co",
                "BIGTIFF=IF_SAFER",
                "-a_nodata",
                str(NODATA_VALUE),
            ]
            run_command(translate_command)
        return output_path
    raise RuntimeError("Unable to create master GeoTIFF; install GDAL (CLI/Python) or rasterio.")


def read_raster_stack(raster_path: Path) -> Tuple[np.ndarray, Tuple[float, float, float, float, float, float], int, int]:
    raster_path = Path(raster_path)
    if HAS_GDAL_PY:
        dataset = gdal.Open(str(raster_path))
        if dataset is None:
            raise RuntimeError(f"Unable to open raster: {raster_path}")
        width = dataset.RasterXSize
        height = dataset.RasterYSize
        transform = dataset.GetGeoTransform()
        data = []
        for band_index in range(dataset.RasterCount):
            band = dataset.GetRasterBand(band_index + 1)
            data.append(band.ReadAsArray().astype(np.float32))
        dataset = None
        return np.stack(data), transform, width, height
    if HAS_RASTERIO:
        with rasterio.open(raster_path) as src:
            data = src.read().astype(np.float32)
            transform = src.transform.to_gdal()
            width = src.width
            height = src.height
        return data, transform, width, height
    raise RuntimeError("Reading rasters requires GDAL or rasterio.")


def build_timeseries_netcdf(
    time_plan: Sequence[Dict[str, object]],
    variables: Sequence[str],
    epsg: int,
    output_path: Path,
) -> None:
    if not time_plan:
        return
    if not HAS_XARRAY:
        print("Skipping NetCDF export because xarray is unavailable.")
        return
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    _, transform, width, height = read_raster_stack(time_plan[0]["path"])
    x_coords = transform[0] + transform[1] * (np.arange(width) + 0.5)
    y_coords = transform[3] + transform[5] * (np.arange(height) + 0.5)
    data_store: Dict[str, List[np.ndarray]] = {var: [] for var in variables}
    times: List[pd.Timestamp] = []
    for entry in time_plan:
        stack, _, _, _ = read_raster_stack(entry["path"])
        if stack.shape[0] < len(variables):
            raise ValueError(f"Raster {entry['path']} has fewer bands than expected.")
        for idx, variable in enumerate(variables):
            data_store[variable].append(stack[idx])
        times.append(pd.to_datetime(entry["date"]))
    coords = {
        "time": pd.to_datetime(times),
        "y": y_coords,
        "x": x_coords,
    }
    data_vars = {
        variable: (("time", "y", "x"), np.stack(arrays, axis=0))
        for variable, arrays in data_store.items()
    }
    dataset = xr.Dataset(data_vars=data_vars, coords=coords, attrs={"crs": f"EPSG:{epsg}", "nodata": NODATA_VALUE})
    dataset.to_netcdf(output_path)
    dataset.close()
    print(f"NetCDF time series created: {output_path}")


def interpolate_with_gdal(
    group_df: pd.DataFrame,
    variables: Sequence[str],
    epsg: int,
    extent: Tuple[float, float, float, float],
    cell_size: float,
    power: float,
    jobs: int,
    output_path: Path,
) -> None:
    if not GDAL_AVAILABLE:
        raise RuntimeError("GDAL CLI tools are not available.")
    grid = build_grid(extent, cell_size)
    env = os.environ.copy()
    env["GDAL_NUM_THREADS"] = str(max(1, int(jobs)))
    with tempfile.TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        subset_path = tmpdir / "points.csv"
        export_cols = ["x", "y", *variables]
        group_df[export_cols].to_csv(subset_path, index=False)
        vrt_path = tmpdir / "points.vrt"
        vrt_path.write_text(create_vrt(subset_path, "points", epsg, "x", "y"), encoding="utf-8")
        per_band_files: List[str] = []
        for variable in tqdm(variables, desc="Variables (GDAL)", leave=False):
            band_path = tmpdir / f"{variable}.tif"
            command = [
                GDAL_CMDS["gdal_grid"],
                "-zfield",
                variable,
                "-a",
                f"invdist:power={power}",
                "-txe",
                str(extent[0]),
                str(extent[2]),
                "-tye",
                str(extent[1]),
                str(extent[3]),
                "-outsize",
                str(grid["width"]),
                str(grid["height"]),
                "-a_srs",
                f"EPSG:{epsg}",
                "-ot",
                "Float32",
                "-of",
                "GTiff",
                "-l",
                "points",
                str(vrt_path),
                str(band_path),
            ]
            run_command(command, env=env)
            per_band_files.append(str(band_path))
        stack_vrt = tmpdir / "stack.vrt"
        run_command(
            [GDAL_CMDS["gdalbuildvrt"], "-separate", str(stack_vrt), *per_band_files],
            env=env,
        )
        translate_command = [
            GDAL_CMDS["gdal_translate"],
            str(stack_vrt),
            str(output_path),
            "-co",
            "COMPRESS=DEFLATE",
            "-co",
            "TILED=YES",
            "-co",
            "BIGTIFF=IF_SAFER",
            "-a_nodata",
            str(NODATA_VALUE),
        ]
        run_command(translate_command, env=env)
    set_band_descriptions(output_path, variables)


def interpolate_with_python(
    group_df: pd.DataFrame,
    variables: Sequence[str],
    epsg: int,
    extent: Tuple[float, float, float, float],
    cell_size: float,
    power: float,
    jobs: int,
    output_path: Path,
) -> None:
    if not HAS_SCIPY:
        raise RuntimeError("SciPy is required for python_idw_kdtree interpolation.")
    grid = build_grid(extent, cell_size)
    coords = group_df[["x", "y"]].to_numpy()
    stack = []
    for variable in tqdm(variables, desc="Variables (Python)", leave=False):
        values = pd.to_numeric(group_df[variable], errors="coerce").to_numpy(dtype=np.float32)
        mask = np.isfinite(values)
        if not mask.any():
            stack.append(np.full((grid["height"], grid["width"]), np.nan, dtype=np.float32))
            continue
        surface = idw_interpolate(
            coords[mask],
            values[mask],
            grid["grid_x"],
            grid["grid_y"],
            power,
            max(1, int(jobs)),
        )
        stack.append(surface.astype(np.float32))
    stack_array = np.stack(stack)
    write_geotiff_stack(stack_array, grid["transform"], epsg, output_path, band_names=list(variables))


def interpolate_with_pykrige(
    group_df: pd.DataFrame,
    variables: Sequence[str],
    epsg: int,
    extent: Tuple[float, float, float, float],
    cell_size: float,
    jobs: int,
    output_path: Path,
) -> None:
    if not HAS_PYKRIGE:
        raise RuntimeError("PyKrige is required for pykrige_ok interpolation.")
    grid = build_grid(extent, cell_size)
    x_coords = grid["x_coords"]
    y_coords = grid["y_coords"]
    stack = []
    for variable in tqdm(variables, desc="Variables (PyKrige)", leave=False):
        values = pd.to_numeric(group_df[variable], errors="coerce").to_numpy(dtype=np.float32)
        mask = np.isfinite(values)
        if mask.sum() < 3:
            stack.append(np.full((grid["height"], grid["width"]), np.nan, dtype=np.float32))
            continue
        try:
            ok = OrdinaryKriging(
                group_df.loc[mask, "x"].to_numpy(),
                group_df.loc[mask, "y"].to_numpy(),
                values[mask],
                variogram_model="spherical",
                verbose=False,
                enable_plotting=False,
                coordinates_type="euclidean",
            )
            surface, _ = ok.execute("grid", x_coords, y_coords, backend="vectorized")
            stack.append(np.asarray(surface, dtype=np.float32))
        except Exception as exc:
            print(f"Kriging failed for {variable}: {exc}")
            stack.append(np.full((grid["height"], grid["width"]), np.nan, dtype=np.float32))
    stack_array = np.stack(stack)
    write_geotiff_stack(stack_array, grid["transform"], epsg, output_path, band_names=list(variables))


def resolve_method(preferred: str) -> str:
    if preferred == "gdal_grid" and not GDAL_AVAILABLE:
        print("GDAL CLI tools were not detected; switching to python_idw_kdtree.")
        return "python_idw_kdtree"
    if preferred == "python_idw_kdtree" and not HAS_SCIPY:
        raise RuntimeError("SciPy is required for python_idw_kdtree.")
    if preferred == "pykrige_ok" and not HAS_PYKRIGE:
        raise RuntimeError("PyKrige is required for pykrige_ok.")
    return preferred


def process_workflow(
    csv_path: str,
    aoi_path: Optional[str],
    lon_col: str,
    lat_col: str,
    date_col: str,
    variable_cols: Sequence[str],
    method: str,
    cell_size: float,
    power: float,
    jobs: int,
    output_folder: str,
) -> None:
    if cell_size <= 0:
        raise ValueError("Cell size must be greater than zero.")
    if power <= 0:
        raise ValueError("IDW power must be positive.")
    if jobs <= 0:
        raise ValueError("Parallel jobs must be at least 1.")
    csv_path = Path(csv_path).expanduser()
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    output_dir = Path(output_folder).expanduser()
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Reading CSV: {csv_path}")
    df = pd.read_csv(csv_path)
    prepared = prepare_dataframe(df, lon_col, lat_col, date_col, variable_cols)
    lon_mean = float(prepared[lon_col].mean())
    lat_mean = float(prepared[lat_col].mean())
    target_epsg = determine_utm_epsg(lon_mean, lat_mean)
    print(f"Target CRS: EPSG:{target_epsg}")
    projected = reproject_dataframe(prepared, lon_col, lat_col, target_epsg)
    extent = derive_extent(projected, cell_size, aoi_path, target_epsg)
    method_to_use = resolve_method(method)
    grouped = list(projected.groupby("__cheaqi_date", sort=True))
    if not grouped:
        raise ValueError("No dated observations were found after filtering.")
    band_records = []
    master_plan = []
    time_series_plan = []
    print(f"Processing {len(grouped)} date group(s) with method '{method_to_use}'.")
    for date_value, group in tqdm(grouped, desc="Dates"):
        label = date_value.strftime("%Y%m%d")
        output_path = output_dir / f"CHEAQI_{label}.tif"
        print(f"Interpolating {label} -> {output_path.name}")
        if method_to_use == "gdal_grid":
            interpolate_with_gdal(group, variable_cols, target_epsg, extent, cell_size, power, jobs, output_path)
        elif method_to_use == "python_idw_kdtree":
            interpolate_with_python(group, variable_cols, target_epsg, extent, cell_size, power, jobs, output_path)
        elif method_to_use == "pykrige_ok":
            interpolate_with_pykrige(group, variable_cols, target_epsg, extent, cell_size, jobs, output_path)
        else:
            raise ValueError(f"Unsupported method: {method_to_use}")
        time_series_plan.append({"date": date_value, "path": str(output_path)})
        for band_index, variable in enumerate(variable_cols, start=1):
            master_plan.append(
                {
                    "path": str(output_path),
                    "band": band_index,
                    "label": f"{label}_{variable}",
                }
            )
            band_records.append(
                {
                    "raster": output_path.name,
                    "band": band_index,
                    "date": date_value.strftime("%Y-%m-%d"),
                    "variable": variable,
                    "master_band": len(master_plan),
                    "master_label": f"{label}_{variable}",
                }
            )
    band_map_path = output_dir / "CHEAQI_per_date_bandmap.csv"
    pd.DataFrame(band_records).to_csv(band_map_path, index=False)
    print(f"Band map written to: {band_map_path}")
    print("Building master GeoTIFF stack...")
    master_output = output_dir / MASTER_STACK_NAME
    try:
        created = create_master_stack(master_plan, master_output)
        if created:
            set_band_descriptions(master_output, [entry["label"] for entry in master_plan])
            print(f"Master stack created: {master_output} ({len(master_plan)} band(s)).")
        else:
            print("No rasters available to build a master stack.")
    except Exception as exc:
        print(f"Master stack creation failed: {exc}")
    netcdf_path = output_dir / NETCDF_TIMESERIES_NAME
    try:
        build_timeseries_netcdf(time_series_plan, variable_cols, target_epsg, netcdf_path)
    except Exception as exc:
        print(f"NetCDF export failed: {exc}")
    print(f"Interpolation complete. Outputs saved to: {output_dir}")


In [ ]:
# Enhanced Interactive CHEAQI Interface with Fixed Action Buttons
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from pathlib import Path
import os
import json

# Enhanced Output Area with better styling
output_area = widgets.Output(
    layout=widgets.Layout(
        border="2px solid #4CAF50", 
        max_height="400px", 
        overflow="auto",
        padding="10px",
        background_color="#f8f9fa"
    )
)

# Status indicator
status_widget = widgets.HTML(
    value="<div style='padding: 10px; background: #e3f2fd; border-radius: 5px; margin: 10px 0;'>"
          "<b>🚀 CHEAQI Interactive Workbench</b><br>"
          "Select your CSV file and configure interpolation parameters below.</div>"
)

# File selection widgets with improved styling
csv_path_widget = widgets.Text(
    description="📁 CSV File:",
    placeholder="Select your environmental data CSV file...",
    layout=widgets.Layout(width="60%"),
    style={'description_width': '120px'}
)
csv_browse_button = widgets.Button(
    description="Browse Files", 
    icon="folder-open",
    button_style="info",
    layout=widgets.Layout(width="140px")
)

aoi_path_widget = widgets.Text(
    description="🗺️ AOI (Optional):",
    placeholder="Optional: Area of Interest shapefile...",
    layout=widgets.Layout(width="60%"),
    style={'description_width': '120px'}
)
aoi_browse_button = widgets.Button(
    description="Browse AOI", 
    icon="map",
    button_style="",
    layout=widgets.Layout(width="140px")
)

# Column selection dropdowns with better labels
lon_dropdown = widgets.Dropdown(
    description="🌍 Longitude:",
    options=[],
    disabled=True,
    layout=widgets.Layout(width="32%"),
    style={'description_width': '100px'}
)
lat_dropdown = widgets.Dropdown(
    description="🌍 Latitude:",
    options=[],
    disabled=True,
    layout=widgets.Layout(width="32%"),
    style={'description_width': '100px'}
)
date_dropdown = widgets.Dropdown(
    description="📅 Date Column:",
    options=[],
    disabled=True,
    layout=widgets.Layout(width="32%"),
    style={'description_width': '100px'}
)

# Enhanced variable selector
variable_select = widgets.SelectMultiple(
    description="📊 Variables:",
    options=[],
    disabled=True,
    rows=8,
    layout=widgets.Layout(width="50%", height="200px"),
    style={'description_width': '100px'}
)

# Variable preview widget
variable_preview = widgets.HTML(
    value="<div style='padding: 10px; background: #f5f5f5; border-radius: 5px;'>"
          "<b>Variable Preview:</b><br>Load CSV to see available variables.</div>",
    layout=widgets.Layout(width="48%", height="200px")
)

# Method and parameter controls with enhanced styling  
method_dropdown = widgets.Dropdown(
    description="⚙️ Method:",
    options=[
        ("🔧 GDAL IDW (Recommended)", "gdal_grid"),
        ("🐍 Python KDTree (Fast)", "python_idw_kdtree"),
        ("📈 PyKrige (Statistical)", "pykrige_ok"),
    ],
    value="gdal_grid",
    layout=widgets.Layout(width="100%"),
    style={'description_width': '100px'}
)

# Parameter controls in organized layout
cell_size_input = widgets.FloatText(
    description="📏 Cell Size (m):",
    value=1000.0,
    min=1.0,
    layout=widgets.Layout(width="48%"),
    style={'description_width': '120px'}
)
power_input = widgets.FloatText(
    description="⚡ IDW Power:",
    value=2.0,
    min=0.1,
    max=10.0,
    layout=widgets.Layout(width="48%"),
    style={'description_width': '120px'}
)

cpu_guess = max(1, os.cpu_count() or 4)
jobs_input = widgets.BoundedIntText(
    description="🔄 CPU Jobs:",
    value=min(4, cpu_guess),
    min=1,
    max=max(1, cpu_guess * 2),
    layout=widgets.Layout(width="48%"),
    style={'description_width': '120px'}
)

# Quality control toggle
quality_check = widgets.Checkbox(
    description="🔍 Enable Quality Assessment",
    value=True,
    layout=widgets.Layout(width="48%"),
    style={'description_width': '200px'}
)

# Output directory selection
output_path_widget = widgets.Text(
    description="📂 Output Dir:",
    placeholder="Select output directory for results...",
    layout=widgets.Layout(width="60%"),
    style={'description_width': '120px'}
)
output_browse_button = widgets.Button(
    description="Browse Folder", 
    icon="folder",
    button_style="",
    layout=widgets.Layout(width="140px")
)

# Action buttons with enhanced styling
load_columns_button = widgets.Button(
    description="🔄 Load & Analyze CSV",
    button_style="info",
    icon="refresh",
    layout=widgets.Layout(width="200px", height="40px")
)

preview_button = widgets.Button(
    description="👀 Preview Selection",
    button_style="",
    icon="eye",
    layout=widgets.Layout(width="200px", height="40px")
)

run_button = widgets.Button(
    description="🚀 Run Interpolation",
    button_style="success",
    icon="play",
    layout=widgets.Layout(width="200px", height="40px")
)

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    orientation='horizontal',
    layout=widgets.Layout(width="100%")
)

def select_path(target_widget: widgets.Text, select_directory: bool, filetypes=None):
    """Enhanced file selection with better error handling"""
    try:
        import tkinter as tk
        from tkinter import filedialog
        
        root = tk.Tk()
        root.withdraw()
        root.attributes("-topmost", True)
        
        if select_directory:
            path = filedialog.askdirectory(title="Select Output Directory")
        else:
            if filetypes is None:
                filetypes = [("CSV files", "*.csv"), ("All files", "*.*")]
            path = filedialog.askopenfilename(
                title="Select CSV File",
                filetypes=filetypes
            )
        
        root.destroy()
        
        if path:
            target_widget.value = path
            with output_area:
                print(f"✅ Selected: {Path(path).name}")
                
    except Exception as exc:
        with output_area:
            print(f"❌ File dialog error: {exc}")
            print("💡 Tip: Manually type the file path in the text field")

def guess_column(columns, keywords):
    """Improved column guessing with pattern matching"""
    lowered = {col.lower(): col for col in columns}
    
    # Exact matches first
    for keyword in keywords:
        if keyword in lowered:
            return lowered[keyword]
    
    # Partial matches
    for col in columns:
        for keyword in keywords:
            if keyword in col.lower():
                return col
    
    return columns[0] if columns else None

def analyze_variables(df, columns):
    """Analyze variables and provide statistics"""
    analysis = {}
    
    for col in columns:
        if col in df.columns:
            series = pd.to_numeric(df[col], errors='coerce')
            analysis[col] = {
                'type': 'numeric' if series.notna().any() else 'text',
                'missing': series.isna().sum(),
                'count': len(series),
                'missing_pct': (series.isna().sum() / len(series) * 100),
                'min': series.min() if series.notna().any() else 'N/A',
                'max': series.max() if series.notna().any() else 'N/A',
                'mean': series.mean() if series.notna().any() else 'N/A'
            }
    
    return analysis

def update_variable_preview(analysis):
    """Update the variable preview with analysis results"""
    if not analysis:
        variable_preview.value = (
            "<div style='padding: 10px; background: #f5f5f5; border-radius: 5px;'>"
            "<b>Variable Preview:</b><br>Load CSV to see available variables.</div>"
        )
        return
    
    html_content = [
        "<div style='padding: 10px; background: #f5f5f5; border-radius: 5px;'>",
        "<b>📊 Variable Analysis:</b><br><br>"
    ]
    
    for var, stats in list(analysis.items())[:10]:  # Show first 10
        if stats['type'] == 'numeric':
            quality = "🟢 Excellent" if stats['missing_pct'] < 5 else "🟡 Good" if stats['missing_pct'] < 20 else "🔴 Poor"
            html_content.append(
                f"<b>{var}:</b><br>"
                f"&nbsp;&nbsp;Quality: {quality} ({stats['missing_pct']:.1f}% missing)<br>"
                f"&nbsp;&nbsp;Range: {stats['min']:.2f} to {stats['max']:.2f}<br><br>"
            )
    
    if len(analysis) > 10:
        html_content.append(f"<i>... and {len(analysis) - 10} more variables</i><br>")
    
    html_content.append("</div>")
    variable_preview.value = "".join(html_content)

# Enhanced event handlers with better feedback
def browse_csv(_):
    select_path(csv_path_widget, False, [("CSV files", "*.csv"), ("All files", "*.*")])

def browse_aoi(_):
    select_path(aoi_path_widget, False, [("Shapefiles", "*.shp"), ("GeoJSON", "*.geojson"), ("All files", "*.*")])

def browse_output(_):
    select_path(output_path_widget, True)

def load_columns(_):
    """Enhanced column loading with comprehensive analysis"""
    csv_path = Path(csv_path_widget.value).expanduser()
    
    with output_area:
        clear_output()
        print("🔍 Analyzing CSV file...")
        
        if not csv_path.exists():
            print(f"❌ CSV not found: {csv_path}")
            status_widget.value = (
                "<div style='padding: 10px; background: #ffebee; border-radius: 5px; margin: 10px 0;'>"
                "<b>❌ Error:</b> CSV file not found. Please check the file path.</div>"
            )
            return
        
        try:
            # Load sample for analysis
            print("📖 Reading CSV structure...")
            sample = pd.read_csv(csv_path, nrows=100)  # Read more for better analysis
            
            if sample.empty:
                print("❌ CSV file is empty")
                return
                
            columns = list(sample.columns)
            print(f"✅ Found {len(columns)} columns")
            
            # Analyze all columns
            analysis = analyze_variables(sample, columns)
            
            # Smart column detection
            lon_guess = guess_column(columns, ["lon", "longitude", "x", "lng"])
            lat_guess = guess_column(columns, ["lat", "latitude", "y"])
            date_guess = guess_column(columns, ["date", "time", "timestamp", "datetime"])
            
            print(f"🎯 Auto-detected columns:")
            print(f"   Longitude: {lon_guess}")
            print(f"   Latitude: {lat_guess}")  
            print(f"   Date: {date_guess}")
            
            # Update dropdowns
            lon_dropdown.options = columns
            lat_dropdown.options = columns
            date_dropdown.options = columns
            
            if lon_guess: lon_dropdown.value = lon_guess
            if lat_guess: lat_dropdown.value = lat_guess
            if date_guess: date_dropdown.value = date_guess
            
            lon_dropdown.disabled = False
            lat_dropdown.disabled = False
            date_dropdown.disabled = False
            
            # Update variable options (exclude coordinate and date columns)
            excluded = {lon_guess, lat_guess, date_guess}
            variable_options = [col for col in columns if col not in excluded and analysis.get(col, {}).get('type') == 'numeric']
            
            variable_select.options = variable_options
            if variable_options:
                # Select first few good quality variables
                good_vars = [var for var in variable_options[:5] if analysis.get(var, {}).get('missing_pct', 100) < 50]
                variable_select.value = tuple(good_vars[:3])  # Select up to 3 good variables
            
            variable_select.disabled = False
            
            # Update preview
            update_variable_preview(analysis)
            
            print(f"✅ Found {len(variable_options)} numeric variables suitable for interpolation")
            
            status_widget.value = (
                "<div style='padding: 10px; background: #e8f5e8; border-radius: 5px; margin: 10px 0;'>"
                f"<b>✅ CSV Loaded:</b> {csv_path.name}<br>"
                f"📊 {len(columns)} total columns, {len(variable_options)} numeric variables<br>"
                f"🎯 Auto-selected: {lon_guess or 'None'} (lon), {lat_guess or 'None'} (lat), {date_guess or 'None'} (date)"
                "</div>"
            )
            
        except Exception as exc:
            print(f"❌ Failed to load CSV: {exc}")
            status_widget.value = (
                "<div style='padding: 10px; background: #ffebee; border-radius: 5px; margin: 10px 0;'>"
                f"<b>❌ Error:</b> Failed to load CSV file<br>{str(exc)}</div>"
            )

def preview_selection(_):
    """Preview the current selection before running interpolation"""
    with output_area:
        clear_output()
        print("👀 SELECTION PREVIEW")
        print("=" * 50)
        
        if not csv_path_widget.value:
            print("❌ No CSV file selected")
            return
            
        if not variable_select.value:
            print("❌ No variables selected")
            return
            
        print(f"📁 Input File: {Path(csv_path_widget.value).name}")
        print(f"🌍 Coordinates: {lon_dropdown.value} (lon) × {lat_dropdown.value} (lat)")
        print(f"📅 Date Column: {date_dropdown.value}")
        print(f"📊 Variables: {len(variable_select.value)} selected")
        
        for i, var in enumerate(variable_select.value, 1):
            print(f"   {i}. {var}")
            
        print(f"⚙️ Method: {dict(method_dropdown.options)[method_dropdown.value]}")
        print(f"📏 Cell Size: {cell_size_input.value} meters")
        print(f"🔄 CPU Jobs: {jobs_input.value}")
        
        if output_path_widget.value:
            print(f"📂 Output: {output_path_widget.value}")
        else:
            print("⚠️ No output directory selected")

def run_clicked(_):
    """Enhanced run function with progress tracking"""
    # Validation
    if not variable_select.value:
        with output_area:
            print("❌ Please select at least one variable")
        status_widget.value = (
            "<div style='padding: 10px; background: #fff3e0; border-radius: 5px; margin: 10px 0;'>"
            "<b>⚠️ Warning:</b> Please select variables for interpolation</div>"
        )
        return
        
    if not output_path_widget.value:
        with output_area:
            print("❌ Please specify an output directory")
        status_widget.value = (
            "<div style='padding: 10px; background: #fff3e0; border-radius: 5px; margin: 10px 0;'>"
            "<b>⚠️ Warning:</b> Please select an output directory</div>"
        )
        return
    
    # Disable controls during processing
    run_button.disabled = True
    preview_button.disabled = True
    progress_bar.value = 0
    
    with output_area:
        clear_output()
        print("🚀 STARTING CHEAQI INTERPOLATION")
        print("=" * 50)
        
        status_widget.value = (
            "<div style='padding: 10px; background: #e3f2fd; border-radius: 5px; margin: 10px 0;'>"
            "<b>🔄 Processing:</b> Running spatial interpolation...</div>"
        )
        
        try:
            progress_bar.value = 20
            print("📊 Processing workflow...")
            
            # Call the main processing function
            process_workflow(
                csv_path=csv_path_widget.value,
                aoi_path=aoi_path_widget.value or None,
                lon_col=lon_dropdown.value,
                lat_col=lat_dropdown.value, 
                date_col=date_dropdown.value,
                variable_cols=list(variable_select.value),
                method=method_dropdown.value,
                cell_size=cell_size_input.value,
                power=power_input.value,
                jobs=jobs_input.value,
                output_folder=output_path_widget.value,
            )
            
            progress_bar.value = 100
            
            status_widget.value = (
                "<div style='padding: 10px; background: #e8f5e8; border-radius: 5px; margin: 10px 0;'>"
                "<b>✅ Success:</b> Interpolation completed successfully!<br>"
                f"📂 Results saved to: {Path(output_path_widget.value).name}</div>"
            )
            
            print("\n" + "=" * 50)
            print("✅ INTERPOLATION COMPLETED SUCCESSFULLY!")
            
        except Exception as exc:
            progress_bar.value = 0
            print(f"\n❌ ERROR: {exc}")
            import traceback
            traceback.print_exc()
            
            status_widget.value = (
                "<div style='padding: 10px; background: #ffebee; border-radius: 5px; margin: 10px 0;'>"
                f"<b>❌ Error:</b> Interpolation failed<br>{str(exc)}</div>"
            )
    
    # Re-enable controls
    run_button.disabled = False
    preview_button.disabled = False

# Connect event handlers
csv_browse_button.on_click(browse_csv)
aoi_browse_button.on_click(browse_aoi)
output_browse_button.on_click(browse_output)
load_columns_button.on_click(load_columns)
preview_button.on_click(preview_selection)
run_button.on_click(run_clicked)

# Create organized layout with better spacing
header_section = widgets.VBox([
    status_widget,
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>📁 File Selection</h3>"),
    widgets.HBox([csv_path_widget, csv_browse_button]),
    widgets.HBox([aoi_path_widget, aoi_browse_button]),
    load_columns_button
])

column_section = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>🎯 Column Configuration</h3>"),
    widgets.HBox([lon_dropdown, lat_dropdown, date_dropdown])
])

variable_section = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>📊 Variable Selection</h3>"),
    widgets.HBox([variable_select, variable_preview])
])

method_section = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>⚙️ Interpolation Settings</h3>"),
    method_dropdown,
    widgets.HBox([cell_size_input, power_input]),
    widgets.HBox([jobs_input, quality_check])
])

output_section = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>📂 Output Configuration</h3>"),
    widgets.HBox([output_path_widget, output_browse_button])
])

action_section = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>🚀 Actions</h3>"),
    widgets.HBox([preview_button, run_button]),
    progress_bar
])

# Main interface layout
main_interface = widgets.VBox([
    widgets.HTML(
        "<div style='text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); "
        "color: white; border-radius: 10px; margin-bottom: 20px;'>"
        "<h1>🌍 CHEAQI Spatial Interpolation Workbench</h1>"
        "<p style='font-size: 16px; margin: 10px 0;'>Interactive Environmental Data Processing & Spatial Analysis</p>"
        "</div>"
    ),
    header_section,
    column_section, 
    variable_section,
    method_section,
    output_section,
    action_section,
    widgets.HTML("<h3 style='color: #1976d2; margin: 20px 0 10px 0;'>📋 Processing Log</h3>"),
    output_area
])

# Display the enhanced interface
display(main_interface)

In [ ]:
def process_workflow(csv_path, aoi_path, lon_col, lat_col, date_col, variable_cols, 
                   method, cell_size=1000, power=2.0, jobs=4, output_folder=None):
    """
    Enhanced processing workflow with proper error handling and progress tracking
    """
    import pandas as pd
    import geopandas as gpd
    from pathlib import Path
    import numpy as np
    from datetime import datetime
    import warnings
    warnings.filterwarnings('ignore')
    
    # Setup paths
    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    
    if output_folder is None:
        output_folder = csv_path.parent / f"cheaqi_outputs_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    print(f"📁 Input: {csv_path.name}")
    print(f"📂 Output: {output_folder}")
    print(f"📊 Variables: {len(variable_cols)} selected")
    print(f"⚙️ Method: {method}")
    
    # Load and validate data
    print("\n🔍 Loading and validating data...")
    df = pd.read_csv(csv_path)
    
    # Check required columns
    missing_cols = []
    if lon_col not in df.columns:
        missing_cols.append(lon_col)
    if lat_col not in df.columns:
        missing_cols.append(lat_col)
    if date_col not in df.columns:
        missing_cols.append(date_col)
    
    for var in variable_cols:
        if var not in df.columns:
            missing_cols.append(var)
    
    if missing_cols:
        raise ValueError(f"Missing columns in CSV: {missing_cols}")
    
    # Clean and validate coordinates
    original_count = len(df)
    df = df.dropna(subset=[lon_col, lat_col])
    
    # Convert coordinates to numeric
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    
    # Remove invalid coordinates
    valid_coords = (
        (df[lon_col] >= -180) & (df[lon_col] <= 180) &
        (df[lat_col] >= -90) & (df[lat_col] <= 90) &
        df[lon_col].notna() & df[lat_col].notna()
    )
    df = df[valid_coords]
    
    cleaned_count = len(df)
    print(f"   📊 Data points: {original_count} → {cleaned_count} (after cleaning)")
    
    if cleaned_count == 0:
        raise ValueError("No valid coordinate data found after cleaning")
    
    # Process each variable
    results_summary = []
    
    for i, variable in enumerate(variable_cols, 1):
        progress_bar.value = int(20 + (i / len(variable_cols)) * 60)  # 20-80% range
        
        print(f"\n🔄 Processing variable {i}/{len(variable_cols)}: {variable}")
        
        try:
            # Clean variable data
            var_df = df[[lon_col, lat_col, date_col, variable]].copy()
            var_df[variable] = pd.to_numeric(var_df[variable], errors='coerce')
            var_df = var_df.dropna(subset=[variable])
            
            if len(var_df) == 0:
                print(f"   ⚠️ No valid data for {variable}")
                continue
                
            # Basic statistics
            stats = {
                'count': len(var_df),
                'min': var_df[variable].min(),
                'max': var_df[variable].max(), 
                'mean': var_df[variable].mean(),
                'std': var_df[variable].std()
            }
            
            print(f"   📈 Stats: {stats['count']} points, range: {stats['min']:.3f} to {stats['max']:.3f}")
            
            # Save processed data
            processed_file = output_folder / f"{variable}_processed_data.csv"
            var_df.to_csv(processed_file, index=False)
            
            # Create interpolation based on method
            output_file = output_folder / f"{variable}_{method}_interpolated.tif"
            
            if method == "gdal_grid":
                result = interpolate_gdal_idw(var_df, lon_col, lat_col, variable, 
                                           output_file, cell_size, power, aoi_path)
            elif method == "python_idw_kdtree":
                result = interpolate_python_kdtree(var_df, lon_col, lat_col, variable,
                                                 output_file, cell_size, power, aoi_path)
            elif method == "pykrige_ok":
                result = interpolate_pykrige_ok(var_df, lon_col, lat_col, variable,
                                              output_file, cell_size, aoi_path)
            else:
                print(f"   ❌ Unknown method: {method}")
                continue
            
            # Store results
            result_info = {
                'variable': variable,
                'method': method,
                'data_points': stats['count'],
                'output_file': output_file.name,
                'stats': stats,
                'success': True
            }
            
            if result:
                result_info.update(result)
            
            results_summary.append(result_info)
            print(f"   ✅ Completed: {output_file.name}")
            
        except Exception as e:
            print(f"   ❌ Failed: {str(e)}")
            results_summary.append({
                'variable': variable,
                'method': method, 
                'error': str(e),
                'success': False
            })
    
    # Generate summary report
    progress_bar.value = 90
    print(f"\n📋 Generating summary report...")
    
    summary_file = output_folder / "interpolation_summary.json"
    
    report = {
        'timestamp': datetime.now().isoformat(),
        'input_file': str(csv_path),
        'output_folder': str(output_folder),
        'method': method,
        'parameters': {
            'cell_size': cell_size,
            'power': power,
            'jobs': jobs
        },
        'coordinate_columns': {
            'longitude': lon_col,
            'latitude': lat_col,
            'date': date_col
        },
        'total_variables': len(variable_cols),
        'successful_interpolations': sum(1 for r in results_summary if r.get('success', False)),
        'failed_interpolations': sum(1 for r in results_summary if not r.get('success', False)),
        'results': results_summary
    }
    
    with open(summary_file, 'w') as f:
        json.dump(report, f, indent=2, default=str)
    
    # Create simple HTML report
    html_file = output_folder / "interpolation_report.html"
    create_html_report(report, html_file)
    
    progress_bar.value = 100
    
    print(f"\n✅ Processing completed!")
    print(f"📊 Success: {report['successful_interpolations']}/{report['total_variables']} variables")
    print(f"📂 Results saved to: {output_folder}")
    print(f"📋 Summary: {summary_file.name}")
    print(f"🌐 Report: {html_file.name}")
    
    return report


def interpolate_gdal_idw(df, lon_col, lat_col, var_col, output_file, cell_size=1000, power=2.0, aoi_path=None):
    """GDAL IDW interpolation with enhanced error handling"""
    try:
        from osgeo import gdal, ogr, osr
        import tempfile
        
        print(f"   🔧 Running GDAL IDW interpolation...")
        
        # Create temporary VRT file
        with tempfile.NamedTemporaryFile(mode='w', suffix='.vrt', delete=False) as vrt_file:
            vrt_content = f"""<OGRVRTDataSource>
    <OGRVRTLayer name="points">
        <SrcDataSource>{df.to_csv(index=False)}</SrcDataSource>
        <GeometryType>wkbPoint</GeometryType>
        <LayerSRS>EPSG:4326</LayerSRS>
        <GeometryField encoding="PointFromColumns" x="{lon_col}" y="{lat_col}"/>
    </OGRVRTLayer>
</OGRVRTDataSource>"""
            vrt_file.write(vrt_content)
            vrt_path = vrt_file.name
        
        # Calculate extent
        bounds = [df[lon_col].min(), df[lat_col].min(), df[lon_col].max(), df[lat_col].max()]
        
        # Build GDAL grid command
        options = gdal.GridOptions(
            algorithm=f'invdist:power={power}',
            outputBounds=bounds,
            width=int((bounds[2] - bounds[0]) * 111320 / cell_size),
            height=int((bounds[3] - bounds[1]) * 111320 / cell_size),
            outputSRS='EPSG:4326',
            layers=['points'],
            zfield=var_col,
            format='GTiff'
        )
        
        # Run interpolation
        gdal.Grid(str(output_file), vrt_path, options=options)
        
        # Cleanup
        Path(vrt_path).unlink(missing_ok=True)
        
        if output_file.exists():
            return {'method_details': f'GDAL IDW (power={power}, cell_size={cell_size}m)'}
        else:
            raise RuntimeError("GDAL interpolation failed - no output file created")
            
    except Exception as e:
        raise RuntimeError(f"GDAL IDW failed: {str(e)}")


def interpolate_python_kdtree(df, lon_col, lat_col, var_col, output_file, cell_size=1000, power=2.0, aoi_path=None):
    """Python-based IDW using KDTree for fast nearest neighbor search"""
    try:
        from sklearn.neighbors import KDTree
        import numpy as np
        from osgeo import gdal, osr
        
        print(f"   🐍 Running Python KDTree IDW interpolation...")
        
        # Prepare data
        coords = df[[lon_col, lat_col]].values
        values = df[var_col].values
        
        # Build KDTree
        tree = KDTree(coords)
        
        # Create grid
        bounds = [df[lon_col].min(), df[lat_col].min(), df[lon_col].max(), df[lat_col].max()]
        
        # Calculate grid dimensions
        width = int((bounds[2] - bounds[0]) * 111320 / cell_size)
        height = int((bounds[3] - bounds[1]) * 111320 / cell_size)
        
        x = np.linspace(bounds[0], bounds[2], width)
        y = np.linspace(bounds[1], bounds[3], height)
        xx, yy = np.meshgrid(x, y)
        
        grid_points = np.column_stack([xx.ravel(), yy.ravel()])
        
        # Find nearest neighbors and interpolate
        k = min(10, len(coords))  # Use up to 10 nearest neighbors
        distances, indices = tree.query(grid_points, k=k)
        
        # IDW calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            weights = 1.0 / np.power(distances, power)
            weights[distances == 0] = 1e16  # Handle exact matches
            
            weighted_values = weights * values[indices]
            interpolated = np.sum(weighted_values, axis=1) / np.sum(weights, axis=1)
        
        # Reshape to grid
        grid = interpolated.reshape(height, width)
        
        # Save as GeoTIFF
        driver = gdal.GetDriverByName('GTiff')
        dataset = driver.Create(str(output_file), width, height, 1, gdal.GDT_Float32)
        
        # Set geotransform
        geotransform = (bounds[0], (bounds[2] - bounds[0]) / width, 0,
                       bounds[3], 0, -(bounds[3] - bounds[1]) / height)
        dataset.SetGeoTransform(geotransform)
        
        # Set projection
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        dataset.SetProjection(srs.ExportToWkt())
        
        # Write data
        band = dataset.GetRasterBand(1)
        band.WriteArray(np.flipud(grid))  # Flip vertically for correct orientation
        band.SetNoDataValue(-9999)
        
        dataset = None  # Close file
        
        return {'method_details': f'Python KDTree IDW (power={power}, neighbors={k})'}
        
    except Exception as e:
        raise RuntimeError(f"Python IDW failed: {str(e)}")


def interpolate_pykrige_ok(df, lon_col, lat_col, var_col, output_file, cell_size=1000, aoi_path=None):
    """PyKrige Ordinary Kriging interpolation"""
    try:
        from pykrige.ok import OrdinaryKriging
        import numpy as np
        from osgeo import gdal, osr
        
        print(f"   📈 Running PyKrige Ordinary Kriging...")
        
        # Prepare data
        x = df[lon_col].values
        y = df[lat_col].values
        z = df[var_col].values
        
        # Create kriging object
        ok = OrdinaryKriging(x, y, z, variogram_model='linear', verbose=False)
        
        # Create grid
        bounds = [x.min(), y.min(), x.max(), y.max()]
        
        # Calculate grid dimensions  
        width = int((bounds[2] - bounds[0]) * 111320 / cell_size)
        height = int((bounds[3] - bounds[1]) * 111320 / cell_size)
        
        grid_x = np.linspace(bounds[0], bounds[2], width)
        grid_y = np.linspace(bounds[1], bounds[3], height)
        
        # Perform kriging
        z_pred, ss = ok.execute('grid', grid_x, grid_y)
        
        # Save as GeoTIFF
        driver = gdal.GetDriverByName('GTiff')
        dataset = driver.Create(str(output_file), width, height, 1, gdal.GDT_Float32)
        
        # Set geotransform
        geotransform = (bounds[0], (bounds[2] - bounds[0]) / width, 0,
                       bounds[3], 0, -(bounds[3] - bounds[1]) / height)
        dataset.SetGeoTransform(geotransform)
        
        # Set projection
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        dataset.SetProjection(srs.ExportToWkt())
        
        # Write data
        band = dataset.GetRasterBand(1)
        band.WriteArray(z_pred)
        band.SetNoDataValue(-9999)
        
        dataset = None  # Close file
        
        return {
            'method_details': f'PyKrige Ordinary Kriging (variogram=linear)',
            'kriging_variance': float(np.mean(ss))
        }
        
    except Exception as e:
        raise RuntimeError(f"PyKrige failed: {str(e)}")


def create_html_report(report, output_file):
    """Create a simple HTML report"""
    html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>CHEAQI Interpolation Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        .header {{ background: #4CAF50; color: white; padding: 20px; border-radius: 5px; }}
        .section {{ margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; }}
        .success {{ color: #4CAF50; }}
        .error {{ color: #f44336; }}
        table {{ border-collapse: collapse; width: 100%; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>🌍 CHEAQI Spatial Interpolation Report</h1>
        <p>Generated: {report['timestamp']}</p>
    </div>
    
    <div class="section">
        <h2>📊 Summary</h2>
        <p><strong>Input File:</strong> {report['input_file']}</p>
        <p><strong>Method:</strong> {report['method']}</p>
        <p><strong>Variables Processed:</strong> {report['total_variables']}</p>
        <p class="success"><strong>Successful:</strong> {report['successful_interpolations']}</p>
        <p class="error"><strong>Failed:</strong> {report['failed_interpolations']}</p>
    </div>
    
    <div class="section">
        <h2>🔧 Parameters</h2>
        <ul>
            <li>Cell Size: {report['parameters']['cell_size']} meters</li>
            <li>IDW Power: {report['parameters']['power']}</li>
            <li>CPU Jobs: {report['parameters']['jobs']}</li>
        </ul>
    </div>
    
    <div class="section">
        <h2>📋 Detailed Results</h2>
        <table>
            <tr>
                <th>Variable</th>
                <th>Status</th>
                <th>Data Points</th>
                <th>Output File</th>
            </tr>
"""
    
    for result in report['results']:
        status = "✅ Success" if result.get('success', False) else "❌ Failed"
        status_class = "success" if result.get('success', False) else "error"
        points = result.get('data_points', 'N/A')
        output = result.get('output_file', result.get('error', 'N/A'))
        
        html_content += f"""
            <tr>
                <td>{result['variable']}</td>
                <td class="{status_class}">{status}</td>
                <td>{points}</td>
                <td>{output}</td>
            </tr>
"""
    
    html_content += """
        </table>
    </div>
</body>
</html>
"""
    
    with open(output_file, 'w') as f:
        f.write(html_content)

print("✅ Enhanced processing functions loaded and ready!")
print("🎯 Action buttons are now fully functional with:")
print("   • Smart file dialogs with error handling")  
print("   • Comprehensive CSV analysis and column detection")
print("   • Real-time variable preview with quality assessment") 
print("   • Progress tracking during interpolation")
print("   • Detailed results summary and HTML reports")

In [1]:
# Quick Test and Instructions
print("🌍 CHEAQI Interactive Workbench Ready!")
print("=" * 60)
print("📋 INSTRUCTIONS:")
print("1. 🚀 Run the cells above to load the interface")
print("2. 📁 Click 'Browse Files' to select your CSV data file")  
print("3. 🔄 Click 'Load & Analyze CSV' to detect columns automatically")
print("4. 📊 Select variables from the list (multiple selection enabled)")
print("5. ⚙️ Choose interpolation method and adjust parameters")
print("6. 📂 Select output directory for results")
print("7. 👀 Use 'Preview Selection' to verify settings")
print("8. 🚀 Click 'Run Interpolation' to start processing")
print()
print("✨ FEATURES:")
print("• Smart column detection (longitude, latitude, date)")
print("• Variable quality assessment with missing data analysis")  
print("• Multiple interpolation methods (GDAL IDW, Python KDTree, PyKrige)")
print("• Real-time progress tracking")
print("• Comprehensive HTML and JSON reports")
print("• Error handling with detailed feedback")
print()
print("🎯 Ready to process environmental data! Select your CSV file to begin.")

# Test that key modules are available
try:
    import ipywidgets
    import pandas as pd
    import numpy as np
    print("✅ All required packages loaded successfully")
except ImportError as e:
    print(f"❌ Missing package: {e}")
    print("💡 Install with: pip install ipywidgets pandas numpy")

🌍 CHEAQI Interactive Workbench Ready!
📋 INSTRUCTIONS:
1. 🚀 Run the cells above to load the interface
2. 📁 Click 'Browse Files' to select your CSV data file
3. 🔄 Click 'Load & Analyze CSV' to detect columns automatically
4. 📊 Select variables from the list (multiple selection enabled)
5. ⚙️ Choose interpolation method and adjust parameters
6. 📂 Select output directory for results
7. 👀 Use 'Preview Selection' to verify settings
8. 🚀 Click 'Run Interpolation' to start processing

✨ FEATURES:
• Smart column detection (longitude, latitude, date)
• Variable quality assessment with missing data analysis
• Multiple interpolation methods (GDAL IDW, Python KDTree, PyKrige)
• Real-time progress tracking
• Comprehensive HTML and JSON reports
• Error handling with detailed feedback

🎯 Ready to process environmental data! Select your CSV file to begin.
✅ All required packages loaded successfully


## Usage & Troubleshooting
1. Use the **Browse** buttons to choose the CSV, optional AOI shapefile, and output directory, then press **Reload Columns**.
2. Pick longitude, latitude, date, and one or more variables, configure interpolation settings (GDAL IDW, Python IDW, or PyKrige OK), and press **Run Interpolation**.
3. Watch the log pane for progress; any Python errors or missing dependency warnings will appear there.
4. After the dated GeoTIFFs are generated, the notebook assembles a master stack (`CHEAQI_master_stack.tif`), writes a NetCDF cube (`CHEAQI_timeseries.nc` when xarray is installed), and records both per-date and master band metadata in `CHEAQI_per_date_bandmap.csv`.
5. If GDAL tools are missing, the notebook automatically falls back to the Python KDTree method (requires SciPy); PyKrige requires the `pykrige` package.
6. When troubleshooting, confirm column names, inspect the AOI CRS, and ensure the output directory is writable.
